In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
import pprint
import matplotlib.pyplot as plt
from citipy import citipy


In [2]:
#Read data from csv file
airports_df = pd.read_csv("Airport_Output.csv", usecols = ["Airport Name","Airport Address", "Lat", "Lng"])

#Delete spaces on both sides of the column's name
airports_df.columns = airports_df.columns.str.replace(' ', '')

#Rename columns
airports_df=airports_df.rename(columns={"Lat": "Latitude", "Lng": "longitude"})

#Give format to the summary table 
#airports_df["Latitude"] = airports_df["Latitude"].map("{:.1f}".format)
#airports_df["longitude"] = airports_df["longitude"].map("{:,.1f}".format)
airports_df=airports_df.round({"Latitude": 0, "longitude": 0})

#Drop duplicates
airports_df= airports_df.drop_duplicates()

#Display Data
airports_df

,Latitude,longitude,AirportName,AirportAddress
0,41.0,-74.0,Newark Liberty International Airport,"3 Brewster Rd, Newark"
1,34.0,-118.0,Los Angeles International Airport,"1 World Way, Los Angeles"
2,42.0,-88.0,O'Hare International Airport,"10000 W O'Hare Ave, Chicago"
3,30.0,-95.0,William P. Hobby Airport,"7800 Airport Blvd, Houston"
4,40.0,-75.0,Philadelphia International Airport,"8000 Essington Ave, Philadelphia"
5,33.0,-112.0,Phoenix Sky Harbor International Airport,"3400 E Sky Harbor Blvd, Phoenix"
6,29.0,-98.0,San Antonio International Airport,"9800 Airport Blvd, San Antonio"
7,33.0,-117.0,San Diego International Airport,"3225 N Harbor Dr, San Diego"
8,33.0,-97.0,Dallas/Fort Worth International Airport,"2400 Aviation Dr, DFW Airport"
9,37.0,-122.0,San Francisco International Airport,San Francisco


In [3]:
types_df = pd.read_csv("cuisine.csv")
types_df


,ethnicity
0,chinese
1,cuban
2,czech
3,french
4,german
5,greek
6,haitian
7,hungarian
8,indian
9,indonesian


In [4]:
# Import API key
gkey="h323GWkBwCy3RkzoJGx19eR-THnde-e6YJ4sAkN-zvZOyPCB35nTwT5EPzgpxzZmnvTkd-bWEPqY8TPyiUlTgiSnRXZI8rj-rJ63QOeUZAvHy5MxB-rvFoRkJ5K9XXYx"

# base url
base_url = "https://api.yelp.com/v3/businesses/search"

# set up api_key dictionary
headers = {
        'Authorization': 'Bearer %s' % gkey,
    }

# set up a parameters dictionary
def s_params(target_loc, target_key, target_type):
    params = {
    "location": target_loc, 
    "term": target_key,
    "categories": target_type,    
}
    return params
def s_query(base_url,headers,params):
    response = requests.request('GET', base_url, headers=headers, params=params).json()
    return response

In [5]:
# Copy any airport address from file airport_df
loc = str(input("In which airport are you located?\n"))

name= []
city=[]
latitude= []
longitude= []
price= []
rating=[]
output=[]


def get_rest(cuisine,loc):
    
    #these are the additional params
    add_params = {"sort_by" : "distance","limit" : 1, "radius" : 1000} 

    #these are the original params
    params = s_params(loc,cuisine,"restaurants, All")

    #add additional params to our params from before
    s1_params = {**params,**add_params}

    s1_results = s_query(base_url,headers,s1_params)
    pprint.pprint(s1_results)
    
    if (s1_results):
        name = s1_results["businesses"][0]["name"]
        city = s1_results["businesses"][0]["location"]["city"]
        latitude = s1_results["businesses"][0]["coordinates"]["latitude"]
        longitude = s1_results["businesses"][0]["coordinates"]["longitude"]
        price = s1_results["businesses"][0]["price"]
        rating=s1_results["businesses"][0]["rating"]
        output = [name,city,latitude,longitude,price, rating] 
    else:
        output = ["NaN","NaN","NaN","NaN","NaN","NaN"]
    
    #print(f"The closest {cuisine.upper()} restaurant is {name} which has a rating of {rating} and is located at: \n {address} \n")   
    
    return output

#this is a long loop so print wait before starting
print("Please, wait and do not desperate, it will take a little time... ")

# use iterrows to iterate through pandas dataframe
# optional: use tqdm for progress bar: tqdm(iterator,total)
# like this: for index, row in tqdm(types_df.iterrows(), total=types_df.shape[0]):

for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = str(row['ethnicity'])
   
    try:
        output = get_rest(restr_type, loc)
        name.append(output[0])
        city.append(output[1])
        latitude.append(output[2])
        longitude.append(output[3])
        price.append(output[4])
        rating.append(output[5])
        
        
    except (KeyError, IndexError):
        output = ["NaN","NaN","NaN","NaN","NaN","NaN"]
        name.append(output[0])
        city.append(output[1])
        latitude.append(output[2])
        longitude.append(output[3])
        price.append(output[4])
        rating.append(output[5])

#print done when done        
print("done!")

In which airport are you located?
3 Brewster Rd, Newark
Please, wait and do not desperate, it will take a little time... 
{'businesses': [{'alias': 'wabi-sabi-gates-c70-99-newark',
                 'categories': [{'alias': 'dimsum', 'title': 'Dim Sum'},
                                {'alias': 'panasian', 'title': 'Pan Asian'},
                                {'alias': 'hotdogs', 'title': 'Fast Food'}],
                 'coordinates': {'latitude': 40.68758, 'longitude': -74.182177},
                 'display_phone': '',
                 'distance': 565.2286949681373,
                 'id': '5R3ICArOKtjmHzS1aoX0Hg',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/JnCR2_rCOLJxxuxTRJYk8Q/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '3 Brewster Rd',
                              'address2': '',
                              'address3': '',
                              'city': 'Newark',
                              'country

{'businesses': [],
 'region': {'center': {'latitude': 40.69039796520161,
                       'longitude': -74.17659759521484}},
 'total': 0}
{'businesses': [{'alias': 'salumeria-prima-newark-4',
                 'categories': [{'alias': 'italian', 'title': 'Italian'},
                                {'alias': 'bagels', 'title': 'Bagels'},
                                {'alias': 'sandwiches', 'title': 'Sandwiches'}],
                 'coordinates': {'latitude': 40.6891378248819,
                                 'longitude': -74.177549068025},
                 'display_phone': '(866) 508-3558',
                 'distance': 161.46048071415734,
                 'id': 'dPeituGq_qgPJILHrm6VuQ',
                 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/dYkSzoQXMuA6pB0BWSd6zQ/o.jpg',
                 'is_closed': False,
                 'location': {'address1': 'Newark Liberty International '
                                          'Airport - 3 Brewster Rd',
               

{'businesses': [],
 'region': {'center': {'latitude': 40.69039796520161,
                       'longitude': -74.17659759521484}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 40.69039796520161,
                       'longitude': -74.17659759521484}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 40.69039796520161,
                       'longitude': -74.17659759521484}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 40.69039796520161,
                       'longitude': -74.17659759521484}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 40.69039796520161,
                       'longitude': -74.17659759521484}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 40.69039796520161,
                       'longitude': -74.17659759521484}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 40.69039796520161,
                       'longitude': -74.17659759521484}},
 'tot

In [6]:
# Create a Dataframe
restaurants_df= pd.DataFrame({
                             "Restaurant": name,
                             "City": city,
                             "Latitude": latitude,
                             "longitude": longitude,                        
                             "price": price,
                             "rating": rating
                            }) 
#Delete spaces on both sides of the column's name
restaurants_df.columns = restaurants_df.columns.str.replace(' ', '')


#Convert data object as float
restaurants_df["Latitude"]=restaurants_df.Latitude.astype(float) 
restaurants_df["longitude"] = restaurants_df.longitude.astype(float)



#Delete Nan Values
restaurants_df.dropna()



,Restaurant,City,Latitude,longitude,price,rating
0,Wabi Sabi - Gates C70-99,Newark,40.687580,-74.182177,$$,2
3,Saison,Newark,40.697015,-74.178539,$$$,2.5
5,Mediterranean Bistro,Newark,40.688489,-74.177030,$$,2
10,Guinness Pub,Newark,40.690477,-74.176500,$$,1.5
12,Salumeria Prima,Newark,40.689138,-74.177549,$$$,2
13,Hamachi Sushi,Newark,40.680595,-74.179886,$$,2
17,QDOBA Mexican Eats,Newark,40.687513,-74.182469,$,2
18,SURF Bar,Newark,40.690462,-74.176790,$$,2
26,Ruby Tuesdays,Newark,40.686391,-74.179752,$$,2


In [7]:
#Give format to the summary table 
#restaurants_df["Latitude"] = restaurants_df["Latitude"].map("{:.1f}".format)
#restaurants_df["longitude"] = restaurants_df["longitude"].map("{:,.1f}".format)
restaurants_df=restaurants_df.round({"Latitude": 0, "longitude": 0})

In [8]:

#Save DataFrame as csv file
restaurants_df.to_csv( "restaurants_yelp.csv")                     


In [9]:
# Combine the data into a single dataset
combined_data = pd.merge(restaurants_df, airports_df, how="outer", on = ["Latitude", "longitude"])

# Display the data table for preview
combined_data.dropna()


,Restaurant,City,Latitude,longitude,price,rating,AirportName,AirportAddress
0,Wabi Sabi - Gates C70-99,Newark,41.0,-74.0,$$,2,Newark Liberty International Airport,"3 Brewster Rd, Newark"
1,Saison,Newark,41.0,-74.0,$$$,2.5,Newark Liberty International Airport,"3 Brewster Rd, Newark"
2,Mediterranean Bistro,Newark,41.0,-74.0,$$,2,Newark Liberty International Airport,"3 Brewster Rd, Newark"
3,Guinness Pub,Newark,41.0,-74.0,$$,1.5,Newark Liberty International Airport,"3 Brewster Rd, Newark"
4,Salumeria Prima,Newark,41.0,-74.0,$$$,2,Newark Liberty International Airport,"3 Brewster Rd, Newark"
5,Hamachi Sushi,Newark,41.0,-74.0,$$,2,Newark Liberty International Airport,"3 Brewster Rd, Newark"
6,QDOBA Mexican Eats,Newark,41.0,-74.0,$,2,Newark Liberty International Airport,"3 Brewster Rd, Newark"
7,SURF Bar,Newark,41.0,-74.0,$$,2,Newark Liberty International Airport,"3 Brewster Rd, Newark"
8,Ruby Tuesdays,Newark,41.0,-74.0,$$,2,Newark Liberty International Airport,"3 Brewster Rd, Newark"


In [10]:
#Save DataFrame as csv file
pd.DataFrame.to_csv(restaurants_df, "restaurants_in _airports.csv")